In [3]:
import pandas as pd
import numpy as np
import string
import re

In [4]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [5]:
test_data.head(5)


In [6]:
train_data.fillna('Unknown', inplace=True)

In [7]:
import string
exclude = set(string.punctuation)



def remove_punctuation(x):
    try:
        x = ''.join(ch for ch in x if ch not in exclude)
    except:
        pass
    return x

In [8]:
train_data.comment_text = train_data.comment_text.apply(remove_punctuation)

In [9]:
train_data['comment_text'].replace(to_replace="\n", value=r" ", regex=True, inplace=True)

In [10]:
train_data = train_data.apply(lambda x: x.astype(str).str.lower())

In [11]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

In [12]:

train_data['Tokenized_words'] = train_data['comment_text'].apply(word_tokenize)

In [13]:
stop_words = set(stopwords.words("english"))

In [14]:
train_data['Keywords_nostopwords']= train_data['Tokenized_words'].apply(lambda x: [item for item in x if item not in stop_words])

In [15]:
train_data.head(5)

In [16]:
#split_data = nltk.pos_tag()
#sent = train_data["Keywords_nostopwords"].apply(split_data)
#[b  for b in sent if b[-1] not in ['CC','CD','DT','EX','IN','LS','MD','PDT','POS','PRP','PRP$','WRB','WP','VBD','VBN','VBP','VB','RB']]


In [17]:
tok_and_tag = lambda x: pos_tag(x)
#tok_and_tag(train_data['Keywords_nostopwords'][0])
train_data["pos_Tag"]=train_data["Keywords_nostopwords"].apply(tok_and_tag)

In [18]:
train_data["comment_text"]

In [52]:
train_context = train_data["comment_text"]
test_context = test_data["comment_text"]


test_context.head(5)

In [53]:
train_context.head(5)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=1,max_features=10000,ngram_range=(1,3),stop_words = 'english')


In [55]:
X_train_dtm = tfidf.fit_transform(train_context)
X_train_dtm
X_test_dtm = tfidf.transform(test_context)
X_test_dtm

In [56]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression



In [57]:
from sklearn.linear_model import SGDClassifier


In [94]:
train_data.head(5)

In [92]:
dataframesample= pd.DataFrame(test_data.id)
label = list(train_data.iloc[:,2:].columns) 
for i in range(2,8):
    lr = SGDClassifier(loss='log', penalty='l1', alpha=1e-06)
    y_train = train_data.iloc[:,i]
    %time lr.fit(X_train_dtm, y_train)
    y_pred_class = lr.predict_proba(X_test_dtm)
    dataframesample[label[i-2]] = (y_pred_class[:,1])
dataframesample.to_csv('output_hackathon.csv',index=False)
